# All pairwise associations between Gene Expression and MicroRNA data in TCGA
Check out more notebooks at our ['Regulome Explorer Repository'](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer)!


```
Title:   All pairwise associations between Gene Expression and MicroRNA data in TCGA
Author:  Boris Aguilar
Created: 03-11-2020
Purpose: Demonstrate how to use the Regulome explorer functionality to find significant associations from Gene expression and microRNA BigQuery tables
URL:     https://github.com/isb-cgc/Community-Notebooks/blob/master/Correlation-GeneExpression-MicroRNA


```
***

In this notebook, we use BigQuery to find significant associations between all possible pairs of genes and unique identifiers available in the RNAseq and microRNA data available in TCGA. The cohort for this analysis consists of BrCA patients with less than 50 years at the time of diagnosis, with II, IIA, IIB pathological stage, race white or black, and excluding patients with hispanic or latino ethnicity.

To find significant associations between RNAseq and microRNA, we used Bigquery to compute Pearson correlation between all possible pairs. The necessary Queries are generated using functions implemented in the [Regulome Explorer module](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module). 

### Authentication
The first step is to authorize access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) and alternative authentication methods can be found [here](https://googleapis.github.io/google-cloud-python/latest/core/auth.html).

In [36]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
from google.cloud import bigquery
import seaborn as sns
from scipy import stats
import re_module.bq_functions as regulome

### Parameters
In this experiment we will use Gene expression data and micro RNA data available in ISB-CGC. The information about the Bigquery tables and their columns are stored in the Features dictionary. The desired significance level and the minimum number of patients for the correlation computations are free parameteres that can be changed.

If you want to test this notebook checking all possible pairs please replace "labels1 = ..." with "labels1 = ' IS NOT NULL '" but know the computations will take several minutes to complete, we recommend to generate the query and copy it in the console.

In [37]:
# information from Bigquery tables
Features = {     'Gene Expression' : { 'table'  : 'isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression',
                                       'symbol' : 'gene_name',
                                       'study'  : 'project_short_name',
                                       'data'   : 'AVG( LOG10( HTSeq__Counts + 1 ) ) ',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',  
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND HTSeq__Counts IS NOT NULL',
                                       'dattype': 'numeric' },
             'MicroRNA Expression': {  'table'  : 'isb-cgc.TCGA_hg38_data_v0.miRNAseq_Expression',
                                       'symbol' : 'mirna_id',
                                        'study' : 'project_short_name',
                                       'data'   : 'AVG( reads_per_million_miRNA_mapped )',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND reads_per_million_miRNA_mapped IS NOT NULL',
                                       'dattype': 'numeric'}
                
               }

# parameteres for Gene expression data
feat1 = Features['Gene Expression']
cohort1 = feat1['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) "  #" IN UNNEST(@PATIENTLIST) "
labels1 = ' = \'IGLVV-66\' '  # ' IS NOT NULL ' use this for all pair-wise comaprison. Change IGLVV-66 for your favorite gene 

# parameteres for Gene expression data
feat2 = Features['MicroRNA Expression'] 
cohort2 = feat2['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) " # " IN UNNEST(@PATIENTLIST) "
labels2 = ' IS NOT NULL '

# minimum correlation
significance_level = 0.01

# minimun number of samples
nsamples = 25

### Read cohort
The cohort for this analysis consists of BrCA patients with less than 50 years at the time of diagnosis, with II, IIA, IIB pathological stage, race white or black, and excluding patients with hispanic or latino ethnicity.

In [38]:
cohort = """
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis <=50 AND
    pathologic_stage IN ('Stage II', 'Stage IIA', 'Stage IIB') AND
    race IN ('WHITE', 'BLACK OR AFRICAN AMERICAN') AND ethnicity = "NOT HISPANIC OR LATINO"
)
"""

### Queries to read tables
The queries to retreive the necessary data, query_t1 for Gene expression and query_t2 for micro RNA, are generated using a function available in the regulome explorer module available in here https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module.  

In [39]:
query_t1 = regulome.generic_numeric_bqtable ( 'table1' , feat1, cohort1, labels1 )
query_t2 = regulome.generic_numeric_bqtable ( 'table2' , feat2, cohort2, labels2 ) 

### Queries to join tables and perform statistics
The two tables are integrated into one table to compute correlations. The query to perform these operations is obtained by using a function implemented in the regulome explorer module.  'sql' contains the complete query to perform the desired correlations.  You can copy the query (which is printed ) to the Bigquery - Google cloud console to run the query and obtain the results. The computations can take 15 minutes. Alternatively you can go to "Run the Query" section of this notebooks. 

Note: `cgc-05-0042.Auxiliary.significance_level_ttest2` is a [persistent function](https://cloud.google.com/bigquery/docs/reference/standard-sql/user-defined-functions)  that estimates upper bounds of p-vaues ( significance levels ), so we can filter out correlations that are not statistically significant ( alpha > significance_level ).

In [40]:
summ_query = regulome.get_summarized_table('Gene Expression',feat1,'MicroRNA Expression',feat2)

sql = ( 'WITH' + cohort + ',' + query_t1 + ',' + query_t2 + ',' + summ_query + """
SELECT *,
   `cgc-05-0042.Auxiliary.significance_level_ttest2`(n-2, ABS(correlation)*SQRT( (n-2)/((1+correlation)*(1-correlation)))) as alpha
FROM summ_table
WHERE n > {0} AND ABS(correlation) < 0.9999
GROUP BY 1,2,3,4,5
HAVING alpha <= {1}
ORDER BY alpha ASC, correlation DESC
""".format( str(nsamples) , str(significance_level) ) )

print( sql )  # THIS Query perform the computations

WITH
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis <=50 AND
    pathologic_stage IN ('Stage II', 'Stage IIA', 'Stage IIB') AND
    race IN ('WHITE', 'BLACK OR AFRICAN AMERICAN') AND ethnicity = "NOT HISPANIC OR LATINO"
)
,
table1 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SELECT
      gene_name AS symbol, 
      AVG( LOG10( HTSeq__Counts + 1 ) )  AS data,
      case_barcode AS ParticipantBarcode
   FROM `isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression`
   WHERE case_barcode IN ( SELECT case_barcode FROM cohort )     # cohort 
         AND gene_name  = 'IGLVV-66'   # labels 
         AND HTSeq__Counts IS NOT NULL  
   GROUP BY
      ParticipantBarcode, symbol
   )
)
,
table2 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SELECT
      mirna_id AS symbol, 
      AVG( reads_per_million_miRNA_mapped ) AS data,
      case_barcode AS ParticipantBarc

### Run the Query
The following steps run BigQuery and generate a table with significant associations.
Note: alpha is an upper bound of the significant level. 

In [41]:
bqclient = bigquery.Client()
df_results = regulome.runQuery ( bqclient, sql, [], [], [], dryRun=False )
df_results


 in runQuery ... 
    the results for this query were previously cached 


,symbol1,symbol2,n,correlation,alpha
0,IGLVV-66,hsa-mir-124-3,135,0.999628,0.0001
1,IGLVV-66,hsa-mir-124-1,135,0.999431,0.0001
2,IGLVV-66,hsa-mir-124-2,135,0.999174,0.0001
3,IGLVV-66,hsa-mir-153-1,135,0.706516,0.0001
4,IGLVV-66,hsa-mir-3166,135,0.666669,0.0001
5,IGLVV-66,hsa-mir-1224,135,0.621718,0.0001
6,IGLVV-66,hsa-mir-4761,135,0.505079,0.0010
7,IGLVV-66,hsa-mir-153-2,135,0.500582,0.0050
